In [1]:
import sys
import scipy
import numpy as np
import matplotlib
import pandas as pd
import sklearn
import io
import requests
from nltk.tokenize import word_tokenize
import nltk
from collections import Counter
from itertools import chain
import re
# from tqdm import tqdm_notebook as tqdm

In [2]:
df = pd.read_csv('interview-data.csv')
raw_names = df['plaid_dirty_name']
unique_names = df.plaid_dirty_name.unique()

In [3]:
raw_names2 = []
for name in raw_names:
#     name = name.replace
    tempName = re.sub('[^a-zA-Z0-9\n\.]', ' ', name)
    tempName = tempName.replace('.', ' ')
    raw_names2.append(tempName)

In [4]:
# TOKENIZING DATA
tokenized_sents = [word_tokenize(name) for name in raw_names2]
total_tokens = 0

# REPLACING NUMBERS AND DIGITS WITH STRING "NUM"
for i, words in enumerate(tokenized_sents):
    total_tokens += len(words)
    for j, word in enumerate(words):
        if word.isnumeric():
            tokenized_sents[i][j] = 'NUM'
        else:
            tokenized_sents[i][j] = word.lower()

In [6]:
# FREQUENCY OF WORD IN WHOLE DATASET
countDict = Counter(chain.from_iterable(tokenized_sents))

In [7]:
words = []                     # unique set of words
word_count = []                # count of each unique word in whole doc
for word in countDict:
    words.append(word)
    word_count.append(countDict[word])

In [8]:
tf = []                        # term frequency
for sentence in tokenized_sents:
    tempSent = []
    for word in words:
        tempSent.append(sentence.count(word))
    tf.append(tempSent)

In [9]:
docFreq = [0]*len(words)       # document frequency of each word
for sent in tokenized_sents:
    for i, word in enumerate(words):
        if word in sent:
            docFreq[i] += 1
            continue

In [10]:
idf = []                       # inverse document frequency of each word
for ele in docFreq:
    idf.append(len(tokenized_sents)/ele)

In [11]:
tf_idf = []                    # term freq * idf
for i,sentences in enumerate(tf):
    tempList = []
    for j,word in enumerate(sentences):
        tempList.append(tf[i][j]*docFreq[j])
    tf_idf.append(tempList)

In [ ]:
# Cosine similarity of each merchant name with every other merchant name
cosine_similarity_matrix = np.zeros(shape=(len(tokenized_sents),len(tokenized_sents)))
for i in range(len(tf_idf)-1):
    for j in range(i+1, len(tf_idf)):
        cosine = 1 - scipy.spatial.distance.cosine(tf_idf[i], tf_idf[j])
        cosine_similarity_matrix[i][j] = cosine
        cosine_similarity_matrix[j][i] = cosine

In [ ]:
# Finding set of unique merchant names based on the threshold cosine similarity value
threshold = 0.9
unique_merchants = []
for i, sentence in enumerate(tokenized_sents):
    insert = 1
    if i>0:
        for j, sent2 in enumerate(cosine_similarity_matrix[i]):
            if j < i and sent2 > threshold:
                insert = 0
                break
    if (insert!=0):
        unique_merchants.append(sentence)

In [ ]:
unique_merchants

In [13]:
# TOKENSIZE DATA
# CHANGE DIGITS TO "NUM" STRING
# FIND TF-IDF
# FIND CORRELATION BETWEEN SENTENCES
# CORRELATION  > 0.8 -> SIMILAR SENTENCES 